In [173]:
%reset -fs

In [143]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.decomposition import TruncatedSVD, NMF

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

from nltk.tokenize import MWETokenizer, word_tokenize
from nltk.tag import pos_tag
from nltk.stem.lancaster import LancasterStemmer

import re

[nltk_data] Downloading package punkt to /Users/solo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/solo/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [144]:
df = pd.read_csv('eu_tweet_df_eda1.csv')

/var/folders/gc/kgfpjt4x1d5c1thdk2sl_2hr0000gn/T/ipykernel_10557/3349357611.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('eu_tweet_df_eda1.csv')


In [145]:
df.head()

,id,created_at,text,user_name,user_screen_name,user_id,user_description,user_location,user_verified,lang,...,in_reply_to_user_id,retweet_count,retweet_id,retweet_screen_name,user_created_at,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_statuses_count
0,866239077045207040,Sun May 21 10:27:44 +0000 2017,Pounding the streets of Redland this morning w...,Green💚🌻Molly,GreenPartyMolly,726372601,Molly Scott Cato 👋 Green Party External Comms ...,South West,False,en,...,NaN,23.0,8.662375e+17,bristol_yg,Mon Jul 30 16:21:34 +0000 2012,27154.0,59652.0,1965.0,854.0,69671.0
1,866232717196840960,Sun May 21 10:02:28 +0000 2017,The UKIP BATTLE BUS is coming; please support ...,Dr Julia Reid PhD,julia_reid,43554444,Former Brexit Party MEP for the SW (2014-19) -...,Calne,True,en,...,NaN,38.0,8.662321e+17,DrTeckKhong,Sat May 30 16:46:20 +0000 2009,62208.0,4773.0,6282.0,242.0,139670.0
2,866236815463862272,Sun May 21 10:18:45 +0000 2017,"Yesterday was 10th European Maritime Day, toda...",Catherine Bearder🇪🇺 🇺🇦🇺🇦🇪🇺🔶,catherinemep,454779231,Lib Dem. Board of IFAW and Council of Unlock D...,Oxford,True,en,...,NaN,21.0,8.662204e+17,EU_MARE,Wed Jan 04 11:13:08 +0000 2012,2705.0,32987.0,9121.0,677.0,36328.0
3,866236868404314112,Sun May 21 10:18:57 +0000 2017,.@theSNP Depute Leader and candidate for Moray...,Ian Hudghton,HudghtonSNP,308677137,SNP President 2005 - 2020. SNP Member of the E...,Scotland,False,en,...,NaN,6.0,8.662227e+17,erikgeddes,Tue May 31 20:22:38 +0000 2011,21.0,5044.0,398.0,205.0,1789.0
4,866233710714257408,Sun May 21 10:06:25 +0000 2017,"@Cacophonic That, and fox hunting and dementia...",Catherine Bearder🇪🇺 🇺🇦🇺🇦🇪🇺🔶,catherinemep,454779231,Lib Dem. Board of IFAW and Council of Unlock D...,Oxford,True,en,...,25067514.0,0.0,NaN,NaN,Wed Jan 04 11:13:08 +0000 2012,2705.0,32987.0,9121.0,677.0,36328.0


In [146]:
df.shape

(3189504, 25)

In [147]:
df['user_description'].isnull().sum()

47826

In [148]:

def strip_punc_lower(txt):
    txt = re.sub(pattern= "[^\w\s]",
            repl="",
            string=txt).lower()
    
    txt = re.sub(pattern= "[0-9]",
            repl="",
            string=txt)
    
    return txt
strip_punc_lower('I am, testing #ALEX > than X 10.')

'i am testing alex  than x '

In [149]:
df['text'] = df['text'].apply(lambda x: strip_punc_lower(x))

In [150]:
df.shape

(3189504, 25)

In [151]:
# def multi_word(txt):
#     mwe_tok = MWETokenizer([('i', 'am'), ('i', 'was'), ('i', 'will'), ('i', 'would')])
#     return  mwe_tok.tokenize(word_tokenize(txt))
# print((multi_word('i am not sure i was, but i will find out if i would')))

In [152]:
# df['text'] = df['text'].apply(lambda x: multi_word(x))

In [153]:
# def join_me(l):
#     return ' '.join(l)
# join_me(['i_am', 'not', 'sure', 'i_was', ',', 'but', 'i_will', 'find', 'out', 'if', 'i_would'])

In [154]:
# df['text'] = df['text'].apply(lambda x: join_me(x))

In [155]:
df.head()

,id,created_at,text,user_name,user_screen_name,user_id,user_description,user_location,user_verified,lang,...,in_reply_to_user_id,retweet_count,retweet_id,retweet_screen_name,user_created_at,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_statuses_count
0,866239077045207040,Sun May 21 10:27:44 +0000 2017,pounding the streets of redland this morning w...,Green💚🌻Molly,GreenPartyMolly,726372601,Molly Scott Cato 👋 Green Party External Comms ...,South West,False,en,...,NaN,23.0,8.662375e+17,bristol_yg,Mon Jul 30 16:21:34 +0000 2012,27154.0,59652.0,1965.0,854.0,69671.0
1,866232717196840960,Sun May 21 10:02:28 +0000 2017,the ukip battle bus is coming please support m...,Dr Julia Reid PhD,julia_reid,43554444,Former Brexit Party MEP for the SW (2014-19) -...,Calne,True,en,...,NaN,38.0,8.662321e+17,DrTeckKhong,Sat May 30 16:46:20 +0000 2009,62208.0,4773.0,6282.0,242.0,139670.0
2,866236815463862272,Sun May 21 10:18:45 +0000 2017,yesterday was th european maritime day today i...,Catherine Bearder🇪🇺 🇺🇦🇺🇦🇪🇺🔶,catherinemep,454779231,Lib Dem. Board of IFAW and Council of Unlock D...,Oxford,True,en,...,NaN,21.0,8.662204e+17,EU_MARE,Wed Jan 04 11:13:08 +0000 2012,2705.0,32987.0,9121.0,677.0,36328.0
3,866236868404314112,Sun May 21 10:18:57 +0000 2017,thesnp depute leader and candidate for moray a...,Ian Hudghton,HudghtonSNP,308677137,SNP President 2005 - 2020. SNP Member of the E...,Scotland,False,en,...,NaN,6.0,8.662227e+17,erikgeddes,Tue May 31 20:22:38 +0000 2011,21.0,5044.0,398.0,205.0,1789.0
4,866233710714257408,Sun May 21 10:06:25 +0000 2017,cacophonic that and fox hunting and dementia t...,Catherine Bearder🇪🇺 🇺🇦🇺🇦🇪🇺🔶,catherinemep,454779231,Lib Dem. Board of IFAW and Council of Unlock D...,Oxford,True,en,...,25067514.0,0.0,NaN,NaN,Wed Jan 04 11:13:08 +0000 2012,2705.0,32987.0,9121.0,677.0,36328.0


In [67]:
tv_30docs = TfidfVectorizer(stop_words='english', min_df=30)
dtm_tv_30docs = tv_30docs.fit_transform(list(df['text']))

In [58]:
# df.text

0          pounding the streets of redland this morning w...
1          the ukip battle bus is coming please support m...
2          yesterday was th european maritime day today i...
3          thesnp depute leader and candidate for moray a...
4          cacophonic that and fox hunting and dementia t...
                                 ...                        
3189499    delighted to welcome borisjohnson to clwydwest...
3189500    stopped in at the churchill united church for ...
3189501                              proest httpstcolhopdtms
3189502    fantastic to join isha jp and niaz at their gr...
3189503    sandy cove acres is hosting a home show and em...
Name: text, Length: 3189504, dtype: object

In [194]:
dtm_tv_30docs.shape


(3189504, 55266)

In [22]:
# list_feat_count = []

# for i in range(0, 102, 2):
#     x = i*1.000
#     min_test = x/10000
#     tv = TfidfVectorizer(stop_words='english', min_df=min_test)
#     dtm_tv = tv.fit_transform(list(df['text']))
#     sh = dtm_tv.shape
#     tup = (min_test, sh[1])
#     list_feat_count.append(tup)
  
#     print(min_test, '    ', sh)

In [23]:
# list_feat_count


In [24]:
type(df['text'])

pandas.core.series.Series

In [25]:
tv_20 = TfidfVectorizer(stop_words='english', min_df=20)
dtm_tv_20 = tv_20.fit_transform(list(df['text']))

In [26]:
dtm_tv_20.shape

(3189504, 72620)

In [200]:
feat_20 = list(tv_20.get_feature_names_out())
# feat_20

In [42]:
lsa = TruncatedSVD(10)
lsa_20 = lsa.fit_transform(dtm_tv_20)

In [45]:
lsa_20.shape

(3189504, 10)

In [46]:
lsa_20

array([[ 5.69869543e-02,  1.61587579e-02,  7.08848383e-02, ...,
        -1.70693279e-02, -2.20676889e-04, -4.49845116e-03],
       [ 4.36182220e-02,  7.04707390e-03,  7.76044410e-03, ...,
        -1.03588302e-05,  2.03604389e-04, -2.52261359e-02],
       [ 7.92515915e-02, -2.97500142e-04,  1.36552853e-02, ...,
         1.44965624e-01, -6.76704383e-02,  7.61164472e-02],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 8.87619280e-02,  1.86727074e-02,  7.47421201e-02, ...,
         4.05390788e-03,  1.81410635e-02,  1.89861569e-03],
       [ 7.09022011e-02,  9.85624337e-03,  5.09395101e-02, ...,
        -1.21090324e-02,  1.47829700e-02, -1.19421734e-02]])

In [27]:
lsa.components_

array([[ 5.98787838e-04,  1.84037445e-03,  5.90739769e-04, ...,
         1.08891602e-03,  1.40450683e-03,  6.33781929e-04],
       [ 2.44019592e-05, -3.68943313e-04,  8.14769262e-05, ...,
        -2.40485269e-04, -5.88626905e-04, -8.91616552e-05],
       [ 2.41419039e-04, -2.21517157e-04,  2.17994725e-04, ...,
        -2.71793361e-05, -3.69218474e-04,  3.55272931e-05],
       [-8.41239539e-05, -3.46939835e-04, -1.36913369e-04, ...,
         6.05024361e-05, -5.56935198e-04, -4.94565852e-05],
       [-3.81681831e-05, -1.65599739e-05, -6.56109208e-05, ...,
         7.43662327e-05, -8.29066814e-05,  8.90397100e-05]])

In [52]:
nmf = NMF(n_components=25, init='random', max_iter=1000)
nmf_20 = nmf.fit_transform(dtm_tv_20)

In [53]:
nmf_20.shape


(3189504, 25)

In [54]:
# Function to display the top n terms in each topic
def display_topics(model, feature_names, no_top_words, topic_names = None): 
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix + 1)
        else:
            print("\nTopic: ", topic_names[ix])
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
    print("\n")
    return model, feature_names, no_top_words

In [73]:
### YOUR CODE HERE
output = display_topics(nmf, tv_20.get_feature_names_out(), 10, topic_names=None)



### OUTPUT THE CONTENTS OF THE VARIABLE
output;


Topic  1
led, ridgesrichmond, alexander_minh, redline, chemicals, infer, respected, myear, heinrich, conormcginn

Topic  2
jennykaynz, rhonddabryant, rcn, lodges, dia, gerrymander, riverfront, ricos, metaphor, keynotes

Topic  3
clodaghm_, repwebster, responsiblebusiness, frankscarpitti, jackson, omnibusbill, auxiliary, dijk, rescued, medias

Topic  4
petrisarvamaa, grantrobertson, newbrunswick, dimitriosgian, carrickmines, bambos, hallways, nfusparliament, evacuating, paranoid

Topic  5
petteriorpo, evacuating, carrickmines, dalgety, grantrobertson, extensive, nfusparliament, hillelneuer, fergusodowd, fatigue

Topic  6
enquiries, jerusalem, illegalwildlifetrade, penetrated, evs, departing, hempstead, realterms, invasion, joc

Topic  7
emphatically, jerusalem, hleisurezone, marvellous, invasion, lissadell, evs, baffles, manifestly, dunphy

Topic  8
podsaveamerica, ria, adulthood, lame, illegalwildlifetrade, medias, repwebster, favoured, maddie, overspent

Topic  9
defuse, progressscot

In [71]:
nmf_25topics = NMF(n_components=25, init='random', max_iter=1000)
nmf_55kwords_25topics  = nmf_25topics.fit_transform(dtm_tv_30docs)

In [75]:

output_55kwords_25topics = display_topics(nmf_25topics, tv_30docs.get_feature_names_out(), 10,
                                          topic_names=None)


Topic  1
amp, women, families, communities, govt, family, friends, protect, rights, future

Topic  2
brexit, deal, theresa, says, mays, tory, britain, hard, mps, referendum

Topic  3
time, long, taking, family, year, come, spend, years, change, hope

Topic  4
need, dont, know, like, help, want, make, think, let, really

Topic  5
people, young, american, country, want, million, say, working, lives, disabled

Topic  6
forward, looking, look, working, seeing, really, meeting, tomorrow, joining, hearing

Topic  7
thank, kind, service, fans, coming, biggest, leadership, sharing, having, supporting

Topic  8
congratulations, team, news, best, deserved, proud, fantastic, award, huge, years

Topic  9
health, care, mental, services, access, insurance, americans, social, public, nhs

Topic  10
thanks, having, coming, email, kind, hi, sharing, lovely, huge, really

Topic  11
im, proud, sure, going, sorry, make, thats, glad, ive, afraid

Topic  12
happy, birthday, year, th, wishing, christmas, fa

In [158]:
import csv

# # note: If you use 'b' for the mode, you will get a TypeError
# # under Python3. You can just use 'w' for Python 3

# with open('ur file.csv','w') as out:
#     csv_out=csv.writer(out)
#     csv_out.writerow(['min_df value','num features'])
#     for row in list_feat_count:
#         csv_out.writerow(row)

#     # You can also do csv_out.writerows(data) instead of the for loop

In [131]:
tv_100docs = TfidfVectorizer(stop_words='english', min_df=100)
dtm_tv_100docs = tv_100docs.fit_transform(list(df['text']))

In [133]:
dtm_tv_100docs.shape

(3189504, 25285)

In [134]:
nmf_25topics = NMF(n_components=25, init='random', max_iter=1000)
nmf_25kwords_25topics  = nmf_25topics.fit_transform(dtm_tv_100docs)

In [135]:

output_25kwords_25topics = display_topics(nmf_25topics, tv_100docs.get_feature_names_out(), 10,
                                          topic_names=None)


Topic  1
amp, families, communities, govt, family, friends, help, future, jobs, protect

Topic  2
work, hard, important, doing, proud, amazing, continue, community, team, excellent

Topic  3
women, right, years, men, ago, country, rights, world, absolutely, violence

Topic  4
good, luck, news, really, morning, best, hear, point, meet, friend

Topic  5
vote, labour, party, mps, yes, parliament, election, mp, deal, tory

Topic  6
forward, looking, look, working, seeing, really, meeting, tomorrow, joining, hearing

Topic  7
just, like, said, got, ive, say, love, voted, days, isnt

Topic  8
im, proud, sure, going, sorry, make, thats, glad, afraid, hear

Topic  9
thanks, having, email, coming, kind, hi, sharing, lovely, huge, hosting

Topic  10
people, young, american, country, want, million, working, say, lives, disabled

Topic  11
day, international, world, national, st, beautiful, family, celebrate, busy, remember

Topic  12
government, tax, read, trump, president, cuts, jobs, families,

In [136]:
nmf_50topics = NMF(n_components=50, init='random', max_iter=1000)
nmf_55kwords_50topics  = nmf_50topics.fit_transform(dtm_tv_30docs)

In [137]:

output_55kwords_50topics = display_topics(nmf_50topics, tv_30docs.get_feature_names_out(), 10,
                                          topic_names=None)


Topic  1
amp, govt, families, communities, family, jobs, friends, security, protect, economy

Topic  2
im, sure, going, sorry, glad, thats, afraid, hear, ive, pleased

Topic  3
work, hard, doing, continue, amazing, excellent, fantastic, team, lets, ensure

Topic  4
eu, uk, trade, citizens, leave, union, rights, customs, market, countries

Topic  5
minister, prime, says, foreign, meeting, ministers, question, said, shadow, trade

Topic  6
parliament, european, debate, mps, mp, welcome, member, union, speech, event

Topic  7
vote, peoples, leave, voted, mps, election, voting, referendum, elections, register

Topic  8
trump, president, house, congress, act, senate, american, law, security, border

Topic  9
right, absolutely, thing, thats, youre, quite, far, rights, wrong, human

Topic  10
change, climate, action, real, energy, plan, future, fight, environment, policy

Topic  11
health, care, mental, services, access, social, insurance, nhs, americans, affordable

Topic  12
help, office, 

In [183]:
from nltk.stem.snowball import SnowballStemmer
snow = SnowballStemmer(language='english')

def lemmitize(tweet):
    lemmitized = []
    words = tweet.split()
    lem = ''
    for w in words:
        l = snow.stem(w)
        lem = lem + l + ' '
        # lm = lemmitized.join()
    return lem

In [184]:
tweet = df.text[0]
tweet
lemmitize(tweet)

'pound the street of redland this morn with londonyg great to have them speak to voter in bristol west with us votemolli httpstcongvxwxla '

In [188]:
# df_test['text'] = df_test['text'].apply(lambda x: snow.stem(word) for word in x)
df['text_lem'] = df.text.apply(lambda x: lemmitize(x))
# df['text']

In [189]:
type(df['text_lem'][0])

str

In [190]:
tv_30docs_lem = TfidfVectorizer(stop_words='english', min_df=30)
dtm_tv_30docs_lem = tv_30docs_lem.fit_transform(list(df['text_lem']))

In [191]:
dtm_tv_30docs_lem.shape

(3189504, 44070)

In [192]:
nmf_50topics = NMF(n_components=50, init='random', max_iter=1000)
nmf_45kwords_lem_50topics  = nmf_50topics.fit_transform(dtm_tv_30docs_lem)

In [193]:
output_45kwords_lem_50topics = display_topics(nmf_50topics, tv_30docs.get_feature_names_out(), 10,
                                          topic_names=None)


Topic  1
alifrance, entitled, dioxide, dwyformeirionnydd, melanielatest, contrary, openletterbot, eastyork, freetobeme, funnelled

Topic  2
sarahjonesmpparliamentuk, inventive, defect, emil, rhiannonv, bruegel_org, geldof, fridaysfuture, sayers, eimear

Topic  3
keithprinceam, ruthcadbury, reprichardneal, accomplishes, debs, muslimban, pick, memorials, acknowledging, eased

Topic  4
machinery, seaford, impediments, contempt, alexiholroyd, jack, chop, billwalkermpp, robins, marriage

Topic  5
bewildered, columeastwood, providence, kellytolhurst, rainbowrose_pes, billmckibben, exacerbate, kurdscampaign, macleans, moedas

Topic  6
quip, richard_kaputt, lord_bilimoria, amermedicalassn, heathrow, moorefieldgaa, bruegel_org, adolescent, lgfa, fence

Topic  7
championsleague, impediments, armagh, salmonds, presiding, conducts, banded, fisheriesbill, members, dirklaabs

Topic  8
mariasherwood, darknessintolight, openletterbot, lees, ourfellowamericans, newshour, charlton, charleston, familesb

In [195]:
nmf_75topics = NMF(n_components=75, init='random', max_iter=1000)
nmf_55kwords_75topics  = nmf_75topics.fit_transform(dtm_tv_30docs)

In [196]:
output_55kwords_75topics = display_topics(nmf_75topics, tv_30docs.get_feature_names_out(), 10,
                                          topic_names=None)


Topic  1
country, service, veterans, honor, communities, families, lives, nation, serve, served

Topic  2
way, long, leading, come, start, life, theres, end, step, lead

Topic  3
amp, govt, security, communities, families, food, st, social, continue, protect

Topic  4
help, families, save, federal, rt, communities, raise, lives, veterans, needs

Topic  5
health, care, mental, services, access, social, insurance, americans, nhs, affordable

Topic  6
great, event, night, job, visit, meeting, discussion, evening, initiative, catch

Topic  7
people, young, american, million, disabled, british, voted, lives, say, living

Topic  8
school, students, children, public, education, schools, funding, high, teachers, parents

Topic  9
trump, president, administration, congress, american, trumps, border, security, national, donald

Topic  10
work, hard, doing, continue, excellent, amazing, lets, pay, ensure, staff

Topic  11
vote, peoples, leave, election, mps, voted, voting, referendum, elections,

In [198]:
type(output_55kwords_75topics[1])

numpy.ndarray